---
# Model Training Notebook

- Tensorflow Object Detection Zoo -> [GitHub](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)

- Huge thanks to **Gilbert Tanner** -> [YouTube](https://www.youtube.com/watch?v=HjiBbChYRDw)

- Labelling images -> [GitHub](https://github.com/tzutalin/labelImg)

## Setup:

In [ ]:
# Install all modules:
!pip install --upgrade pip
!pip install --upgrade utils
# !pip install tensorflow
!pip install tensorflow==1.15
!pip install object_detection
!pip install --upgrade protobuf 

In [ ]:
import tensorflow
import os
print(tensorflow.__version__)

---
Turn on GPU Acceleration if you want to train your model. GPU Acces is limited!

First, you'll need to enable GPUs for the notebook:
- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [36]:
# TO BE TESTED
import tensorflow as tf2
device_name = tf2.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Download models-master from GitHub:
% cd /content/
if not os.path.isdir('models-master'):
  !wget https://github.com/tensorflow/models/archive/master.zip
  !unzip /content/master.zip
  %rm -rf master.zip
  pass  # models-master

# Download other needed files:
if not os.path.isdir('models-master/research/object_detection/Templates'):
  !wget https://github.com/WasuMrTomass0/Tutorials/archive/master.zip
  !unzip /content/master.zip
  !cp -a /content/Tutorials-master/CustomObjDetModelTrainingColab/pyScripts/* /content/models-master/research/object_detection/
  %rm -rf master.zip
  %rm -rf /content/Tutorials-master
  pass

In [ ]:
# Connect Your Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Directories setup:
- images/test -  should contain ~30% of all images
- images/train -  should contain ~70% of all images

In [ ]:
%cd /content/models-master/research/object_detection
%mkdir images
%mkdir images/test
%mkdir images/train

%mkdir training

In [6]:
# Copy images from google drive:
# Must contain 'train' and 'test' subdirectories with images and labels.
GOOGLE_DRIVE_IMAGES_PATH = '/content/drive/MyDrive/ML_CSGO/images_v2'

!cp -a {GOOGLE_DRIVE_IMAGES_PATH}/test/* /content/models-master/research/object_detection/images/test/
!cp -a {GOOGLE_DRIVE_IMAGES_PATH}/train/* /content/models-master/research/object_detection/images/train/

In [7]:
# Copy 'classes.txt'
GOOGLE_DRIVE_CLASSES_PATH = '/content/drive/MyDrive/ML_CSGO'
!cp -a {GOOGLE_DRIVE_CLASSES_PATH}/classes.txt /content/models-master/research/object_detection/

In [ ]:
# or move it manually to: /content and then copy it to /content/models-master/research/object_detection by running this cell
!cp -a /content/classes.txt /content/models-master/research/object_detection/

## Generators:

In [ ]:
# Generate files from classes.txt
%cd /content/models-master/research/object_detection
!python GenerateClassesInfo.py

!cp -a labelmap.pbtxt training/
!cp -a label_map.pbtxt training/

In [ ]:
# Convert xml files to csv
!python xml_to_csv.py

In [ ]:
# Generate test and train record
!python generate_tfrecord.py --csv_input=images/test_labels.csv --output_path=test.record --image_dir=images/test
!python generate_tfrecord.py --csv_input=images/train_labels.csv --output_path=train.record --image_dir=images/train

## Configure model, paths and number of classes:


If you want to use model from zoo change link below.

In [12]:
LINK_TO_MODEL_FROM_ZOO = 'http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz'

# Get names from link:
MODEL_NAME_TAR_GZ = LINK_TO_MODEL_FROM_ZOO.split('/')[-1]
MODEL_NAME = MODEL_NAME_TAR_GZ.split('.')[0]

In [ ]:
# Download model from zoo:
%cd /content/models-master/research/object_detection
!wget {LINK_TO_MODEL_FROM_ZOO}
!tar -xzvf {MODEL_NAME_TAR_GZ}
%rm -rf {MODEL_NAME_TAR_GZ}

In [14]:
# Copy config file from /samples to /training
!cp -a /content/models-master/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config /content/models-master/research/object_detection/training/

# Tell me what is the name of config u have choosen:
# for example: 'faster_rcnn_inception_v2_pets'
MODEL_CONFIG_FILE_NAME = 'faster_rcnn_inception_v2_pets' + '.config'

Number of classes:
```
model {
  faster_rcnn {
    num_classes: 37   <----  Change this number!   
  ...
  }
}
```

In [ ]:
# Follow instructions which are displayed after running this cell
print('Navigate to "/content/models-master/research/object_detection/training" \nand open config that was copied from /samples')

print('1', '- ' * 50)

print('Find line: (Around 106)\n\t"fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/model.ckpt"')
print('Swap "PATH_TO_BE_CONFIGURED/model.ckpt" to:\n\t "{}"'.format('/content/models-master/research/object_detection/' + MODEL_NAME + '/model.ckpt'))

print('2', '- ' * 50)

print('Find line: (Around 123 - under "train_input_reader")\n\t"tf_record_input_reader {\n\t\tinput_path: "PATH_TO_BE_CONFIGURED/pet_faces_train.record-?????-of-00010"')
print('Swap "PATH_TO_BE_CONFIGURED/pet_faces_train.record-?????-of-00010" to:\n\t "{}"'.format('/content/models-master/research/object_detection/train.record'))

print('3', '- ' * 50)

print('Find line: (Around 125 - under "train_input_reader")\n\t"label_map_path: "PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt"')
print('Swap "PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt" to:\n\t "{}"'.format('/content/models-master/research/object_detection/training/labelmap.pbtxt'))

print('4', '- ' * 50)

print('Find line: (Around 135 - under "train_input_reader")\n\t"tf_record_input_reader {\n\t\tinput_path: "PATH_TO_BE_CONFIGURED/pet_faces_val.record-?????-of-00010"')
print('Swap "PATH_TO_BE_CONFIGURED/pet_faces_train.record-?????-of-00010" to:\n\t "{}"'.format('/content/models-master/research/object_detection/test.record'))

print('5', '- ' * 50)

print('Find line: (Around 137 - under "train_input_reader")\n\t"label_map_path: "PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt"')
print('Swap "PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt" to:\n\t "{}"'.format('/content/models-master/research/object_detection/training/labelmap.pbtxt'))

# TODO Add automatic replacement

# Training:

In [17]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start 
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.865956143869294


In [ ]:
%cd /content/models-master/research/object_detection
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/models-master/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models-master/research/:/content/models-master/research/slim'

!python setup.py build
!python setup.py install

In [ ]:
!pip install tf_slim
%cd  /content/models-master/research/object_detection/
os.environ['PYTHONPATH'] += ':/content/models-master/research/:/content/models-master/research/slim'

In [ ]:
%cd /content/models-master/research/object_detection
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/{MODEL_CONFIG_FILE_NAME}

# Export trained model:

In [ ]:
%cd /content/models-master/research/object_detection
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-61118 --output_directory new_graph



In [ ]:
%cd /content/models-master/research/object_detection
# Input trained model counter - navigate to "/content/models-master/research/object_detection/training"
# and copy the highest number
counter = 61118

TRAINED_MODEL_DIR_NAME = 'my_trained_model'

# Save directory must be empty/ not created
# %rm -rf {TRAINED_MODEL_DIR_NAME}

# Exporting trained model:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/{MODEL_CONFIG_FILE_NAME} --trained_checkpoint_prefix training/model.ckpt-{counter} --output_directory {TRAINED_MODEL_DIR_NAME}


# Zip new_graph
%cd /content
!zip -r {TRAINED_MODEL_DIR_NAME}.zip /content/models-master/research/object_detection/{TRAINED_MODEL_DIR_NAME}/
!cp -a /content/models-master/research/object_detection/{TRAINED_MODEL_DIR_NAME}/ /content/


In [35]:
# Save to Google Drive
PATH_TO_SAVE_DIR = '/content/drive/MyDrive/ML_CSGO/my_new_graph_27112020' + '/'

!cp -r {TRAINED_MODEL_DIR_NAME}.zip {PATH_TO_SAVE_DIR}
!cp -r {TRAINED_MODEL_DIR_NAME} {PATH_TO_SAVE_DIR}
